
# STRUCTURE

In [1]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp

In [2]:
print(ipa.__version__)
print(toyplot.__version__)

0.9.40
0.18.0


In [3]:
## look for running ipcluster instance, and create load-balancer
ipyclient = ipp.Client()
print("{} engines found".format(len(ipyclient)))

24 engines found


In [4]:
# the path to your HDF5 formatted snps file
data = "./H_st_11rm.snps.hdf5"

In [58]:
imap = {
#"ref": ["reference"],
"Puru": ["H_pe_A24346_pu"],
"JiGu": [ "H_oc_A311_jigu", "H_oc_T355_jigu","H_oc_T15847_jigu"],
"Mach": [ "H_ro_A547_ma", "H_ro_J213_ma", "H_ro_J305_ma", "H_ro_J508_ma", "H_ro_J774_ma", "H_ro_J775_ma", "H_ro_J796_ma", "H_ro_T1842_ma", "H_ro_T366_ma", "H_ro_T385_ma", "H_ro_T471_ma"],#"H_ro_A8296_ma",
"Roar": ["H_st_J364_roar", "H_st_J621_roar", "H_st_J664_roar", "H_st_J665_roar", "H_st_J711_roar", "H_st_J762_roar", "H_st_J765_roar", "H_st_J368_roar", "H_st_J370_roar", "H_st_J374_roar", "H_st_J408_roar"],
"ArSu": ["H_ro_A409_ma", "H_ro_A551_ma", "H_ro_A410_ma", "H_ro_A521_ma","H_st_77860_arsu", "H_st_78249_arsu", "H_st_J525_arsu", "H_st_J530_arsu", "H_st_J536_arsu", "H_st_J572_arsu", "H_st_80727_arsu", "H_st_80800_arsu", "H_st_A272_arsu", "H_st_A273_arsu","H_st_81143_arsu","H_st_85680_arsu", "H_st_85970_arsu"],
"SuTa": ["H_st_T10207_suta", "H_st_T11900_suta", "H_st_A9955_pa", "H_st_T12194_suta","H_st_A4899_suta","H_st_81279_suta",  "H_st_86405_suta","H_st_A14546_suta","H_st_T24564_suta", "H_st_T7114_suta"],
"Para": ["H_st_A16571_pa","H_st_A7597_pa", "H_st_T16744_pa", "H_st_T17858_pa","H_st_A11597_pa", "H_st_A15208_pa", "H_st_A15210_pa"]
}

# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [6]:
# init analysis object with input data and (optional) parameter options
struct = ipa.structure(
    name="H_st_11rm",
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.95,
)

Samples: 60
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 44386
Filtered (mincov): 1654580
Filtered (minmap): 1521392
Filtered (combined): 1662037
Sites after filtering: 40624
Sites containing missing values: 37657 (92.70%)
Missing values in SNP matrix: 84418 (3.46%)


In [10]:
struct.write_structure_files(abs)

('/array1/lmusher/rio_roosevelt_outfiles/H_st_11rm_outfiles/analysis-structure/tmp-H_st_11rm-<built-in function abs>-1.mainparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/H_st_11rm_outfiles/analysis-structure/tmp-H_st_11rm-<built-in function abs>-1.extraparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/H_st_11rm_outfiles/analysis-structure/tmp-H_st_11rm-<built-in function abs>-1.strfile.txt')

In [7]:
struct.mainparams.burnin = 50000
struct.mainparams.numreps = 250000

In [ ]:
struct.run(nreps=10, kpop=[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ipyclient=ipyclient, force=True,)

Parallel connection | amnh-gen-001.internal.amnh.org: 24 cores
[####                ]  21% 3:53:59 | running 110 structure jobs 

In [6]:
struct = ipa.structure(
    data=data, 
    name="H_st_5rm", 
    workdir="analysis-structure",
    imap=imap,
    load_only=True,
)

110 previous results loaded for run [H_st_5rm]


In [7]:
etable = struct.get_evanno_table([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
etable

,Nreps,lnPK,lnPPK,deltaK,estLnProbMean,estLnProbStdev
2,10,0.00,0.00,0.000000,-40885.92,601.724292
3,10,-17.28,487.26,0.685310,-40903.20,711.006545
4,10,469.98,6350.72,8.910024,-40433.22,712.761241
5,10,-5880.74,139014.61,6.947060,-46313.96,20010.568017
6,10,-144895.35,262099.56,1.208290,-191209.31,216917.778707
7,10,117204.21,107732.85,2.805977,-74005.10,38394.055809
8,10,9471.36,105705.70,2.241206,-64533.74,47164.658198
9,10,-96234.34,182579.87,1.498032,-160768.08,121879.837624
10,10,86345.53,92690.80,2.594671,-74422.55,35723.523540
11,10,-6345.27,12919.02,0.354593,-80767.82,36433.335253


In [8]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t3c11b50785954b75bd966933c842153d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 6 7 8 9 10 11 12 K (N ancestral populations) 50000 100000 150000 200000 estLnProbMean 0 4 8 12 deltaK

In [9]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [10]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [11]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t666c9bc56e9b4bf4846bcd4fe17b49be" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_T15847_jigu H_pe_A24346_pu H_oc_A311_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_J364_roar H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_ro_A409_ma H_ro_A551_ma H_ro_A410_ma H_ro_A521_ma H_st_77860_arsu H_st_78249_arsu H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_80727_arsu H_st_80800_arsu H_st_A272_arsu H_st_A273_arsu H_st_81143_arsu H_st_85680_arsu H_st_85970_arsu H_st_T10207_suta H_st_T11900_suta H_st_A9955_pa H_st_T12194_suta H_st_A4899_suta H_st_81279_suta H_st_86405_suta H_st_A14546_suta H_st_T24564_suta H_st_T7114_suta H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa 0.0 0.5 1.0

In [12]:
k = 3
table = struct.get_clumpp_table(k)
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K3] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="td8cf245690564662a8f43271e099ae90" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta 0.0 0.5 1.0

In [17]:
k = 4
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t67ce596fe7c9416081045fdcaf4c3ae2" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_T15847_jigu H_pe_A24346_pu H_oc_A311_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_J364_roar H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_ro_A409_ma H_ro_A551_ma H_ro_A410_ma H_ro_A521_ma H_st_77860_arsu H_st_78249_arsu H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_80727_arsu H_st_80800_arsu H_st_A272_arsu H_st_A273_arsu H_st_81143_arsu H_st_85680_arsu H_st_85970_arsu H_st_T10207_suta H_st_T11900_suta H_st_A9955_pa H_st_T12194_suta H_st_A4899_suta H_st_81279_suta H_st_86405_suta H_st_A14546_suta H_st_T24564_suta H_st_T7114_suta H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa 0.0 0.5 1.0

In [18]:
k = 5
table = struct.get_clumpp_table(k)
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K5] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t784cfbb7e5b547dcac79bda1da1ffc79" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta 0.0 0.5 1.0

In [15]:
k = 6
table = struct.get_clumpp_table(k)
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K6] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="tdd71aa9ce18e4e9b937920d7800a2346" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta 0.0 0.5 1.0

# PCA

In [18]:
#!conda install scikit-learn -c conda-forge -y

In [6]:
import ipyrad.analysis as ipa
import pandas as pd
import toyplot

In [81]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
    
)

Samples: 60
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 44386
Filtered (mincov): 993993
Filtered (minmap): 1699584
Filtered (combined): 1699694
Sites after filtering: 2967
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [82]:
# run the PCA analysis
pca.run()

Subsampling SNPs: 346/2967


In [83]:
pca.draw()

(<toyplot.canvas.Canvas at 0x2b619710ee50>,
 <toyplot.mark.Point at 0x2b61970425d0>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="te563a098dc294113b0686a259369db5a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -2 0 2 4 PC0 (18.6%) explained -2 -1 0 1 2 3 PC1 (7.5%) explained Para Puru Roar ArSu SuTa JiGu Mach

In [23]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("H_st_pca_analysis.csv")

# show the first ten samples and the first 10 PC axes
df.iloc[:10, :10].round(2)

,0,1,2,3,4,5,6,7,8,9
H_oc_A311_jigu,8.57,1.51,26.92,4.94,-15.60,-2.73,0.82,-0.67,2.62,2.90
H_oc_T15847_jigu,2.33,3.56,30.28,7.12,-14.93,-0.56,-0.65,0.56,-2.91,-3.18
H_oc_T355_jigu,23.29,-3.45,1.89,0.11,-4.09,-4.03,4.95,0.36,2.99,3.03
H_pe_A24346_pu,1.81,4.20,26.24,0.16,35.02,1.36,1.92,0.37,0.28,-0.14
H_ro_A409_ma,-8.19,-1.53,-2.07,-1.98,-0.79,-2.89,8.73,-2.07,-3.22,-5.23
H_ro_A410_ma,-8.42,-2.82,-0.42,-1.94,-1.18,4.05,-0.15,-7.86,-2.08,-1.38
H_ro_A521_ma,-8.45,-2.71,-0.15,-2.37,-0.75,4.11,1.08,-6.30,0.63,3.87
H_ro_A547_ma,24.73,-4.71,-3.30,-0.66,0.28,-2.79,3.84,1.33,-0.85,-0.68
H_ro_A551_ma,-8.78,-2.68,-1.25,-2.90,-0.35,5.44,0.92,-7.70,0.72,2.64
H_ro_J213_ma,24.40,-5.00,-3.05,-0.92,0.10,-2.78,4.10,0.83,-0.42,-1.02


# Subsampling with replication

In [24]:
# plot PC axes 0 and 2 with many replicate subsamples
pca.run(nreplicates=25, seed=12345)
pca.draw(0, 1);

Subsampling SNPs: 14857/174582


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t7d3b04dd48a5468a8a5ef8c9634a8ec0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <circle r="

# Kmeans imputation (integer)

In [25]:
# kmeans imputation 
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method=7,
)

# run and draw results for kmeans clustering into 7 groups
pca3.run(nreplicates=25, seed=123)
pca3.draw(0, 1);

Kmeans clustering: iter=0, K=7, mincov=0.9, minmap={'global': 0.5}
Samples: 60
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 44386
Filtered (mincov): 1570811
Filtered (minmap): 993993
Filtered (combined): 1575512
Sites after filtering: 127149
Sites containing missing values: 124182 (97.67%)
Missing values in SNP matrix: 507776 (6.66%)
Imputation: 'sampled'; (0, 1, 2) = 59.5%, 4.9%, 35.5%
{0: ['H_oc_T355_jigu', 'H_ro_A547_ma', 'H_ro_J213_ma', 'H_ro_J305_ma', 'H_ro_J508_ma', 'H_ro_J774_ma', 'H_ro_J775_ma', 'H_ro_J796_ma', 'H_ro_T1842_ma', 'H_ro_T366_ma', 'H_ro_T385_ma', 'H_ro_T471_ma'], 1: ['H_st_T17858_pa'], 2: ['H_pe_A24346_pu', 'H_ro_A410_ma', 'H_ro_A521_ma', 'H_ro_A551_ma', 'H_st_78249_arsu', 'H_st_80800_arsu', 'H_st_81279_suta', 'H_st_85970_arsu', 'H_st_86405_suta', 'H_st_A14546_suta', 'H_st_A273_arsu', 'H_st_A4899_suta', 'H_st_A9955_pa', 'H_st_J374_roar', 'H_st_J525_arsu', 'H_st_J530_arsu', 'H_st_J536_arsu', 'H_st_J572_arsu', 'H_st_T11900_suta', 'H_st_T1

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t05b0707f10124cd69240190f7d20f146" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(99.7%,91.3%,60.3%);fill-opacity:0.036000000000000004;opacity:1.0;stroke:none" transform="tran

In [26]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca3.draw(0, 2)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "H_st_PCA-kmeans-7.pdf")

# T-SNE (ASSESSING COMPONENT LOADINGS)

In [8]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [84]:
pca.run_tsne(subsample=True, perplexity=5.0, n_iter=1000000, seed=423)
pca.draw(size=8);

Subsampling SNPs: 346/2967


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t738daa1cda6d4f5e96ccc9809e454b48" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A547_ma H_ro_A551_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A11597_pa H_st_A14546_suta H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A7597_pa H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T16744_pa H_st_T17858_pa H_st_T24564_suta H_st_T7114_suta -600 -300 0 300 TNSE component 1 -500 0 500 1000 1500 TNSE component 2 Para Puru Roar ArSu SuTa JiGu Mach

In [86]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("H_st_TSNE.csv")

# show the first ten samples and the first 10 PC axes
df.head(10)

,0,1
H_oc_A311_jigu,-339.238,1098.892
H_oc_T15847_jigu,-317.195,1118.517
H_oc_T355_jigu,-488.621,1276.253
H_pe_A24346_pu,149.063,-389.573
H_ro_A409_ma,-177.118,-266.285
H_ro_A410_ma,44.392,40.326
H_ro_A521_ma,118.422,-151.795
H_ro_A547_ma,-387.196,1253.583
H_ro_A551_ma,-64.629,-196.190
H_ro_J213_ma,-463.450,1321.081


In [45]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca.draw(size=8)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "H_st_TSNE_perp6.pdf")

# GENETIC DISTNANCES

In [5]:
import ipyrad.analysis as ipa
import toyplot
ipa.

In [6]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
    subsample_snps=False,
    
)
dist.run()

Samples: 60
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 44386
Filtered (mincov): 993993
Filtered (minmap): 1521392
Filtered (combined): 1528079
Sites after filtering: 174582
Sites containing missing values: 171615 (98.30%)
Missing values in SNP matrix: 1255077 (11.98%)
Imputation: 'sampled'; (0, 1, 2) = 58.1%, 4.2%, 37.7%


In [33]:
# save to a CSV file
dist.dists.to_csv("H_st_distances.csv")

In [34]:
# save to a CSV file with no labels (eems style)
dist.dists.to_csv(
    "H_st_distances_eems.csv",
    header=None,
    index=False,
    sep=" ",
)

In [35]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

In [36]:
toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="ted66cfbe3ea34f4cb2005ccdf2b3a357" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.210079 0.097009 0.180706 0.190083 0.189619 0.185873 0.185242 0.186497 0.185277 0.189206 0.185575 0.183094 0.187161 0.182361 0.173523 0.172584 0.177773 0.173065 0.173695 0.176834 0.172022 0.175883 0.173248 0.173958 0.172297 0.177538 0.174187 0.172360 0.171856 0.177647 0.172652 0.173792 0.171318 0.173065 0.176347 0.178988 0.173781 0.178655 0.173718 0.178787 0.171272 0.171621 0.176570 0.170951 0.172658 0.173437 0.173093 0.173466 0.172211 0.177246 0.172486 0.173248 0.180133 0.185071 0.197529 0.185489 0.190300 0.190266 0.186331 H_oc_T15847_jigu 1 0.210079 0.000000 0.213218 0.218797 0.219295 0.219267 0.215773 0.214169 0.216420 0.215085 0.220011 0.215360 0.213934 0.216832 0.214117 0.200788 0.198405 0.203240 0.199230 0.199047 0.202598 0.200021 0.202621 0.201716 0.199597 0.200467 0.200359 0.197855 0.195009 0.196154 0.199551 0.196630 0.197082 0.196120 0.197959 0.197048 0.202415 0.196819 0.203286 0.197340 0.201813 0.195616 0.197386 0.200994 0.195387 0.195948 0.197128 0.196590 0.196515 0.195501 0.199161 0.195696 0.197180 0.202747 0.210193 0.221529 0.209569 0.214140 0.214129 0.211018 H_pe_A24346_pu 2 0.097009 0.213218 0.000000 0.132442 0.162588 0.161632 0.160269 0.159730 0.161122 0.160016 0.162124 0.161048 0.157571 0.159427 0.157490 0.173099 0.171965 0.175436 0.171770 0.172618 0.175196 0.172813 0.174692 0.172698 0.173603 0.172641 0.174411 0.171839 0.170974 0.171249 0.173993 0.171690 0.172269 0.170161 0.172572 0.173311 0.175723 0.172612 0.176708 0.172091 0.176335 0.171661 0.171301 0.173466 0.171822 0.170814 0.172750 0.172005 0.172595 0.171226 0.173661 0.171936 0.171931 0.176903 0.179297 0.191549 0.179280 0.183117 0.183553 0.181176 H_oc_A311_jigu 3 0.180706 0.218797 0.132442 0.000000 0.074549 0.073765 0.076858 0.077293 0.078089 0.079057 0.073203 0.080512 0.075386 0.072442 0.076612 0.160601 0.158287 0.155331 0.157267 0.159283 0.155549 0.159066 0.156522 0.159616 0.158115 0.158630 0.153504 0.157347 0.156780 0.157582 0.154002 0.156167 0.158900 0.157639 0.158997 0.155073 0.155721 0.158613 0.155113 0.158711 0.156528 0.158636 0.157817 0.153727 0.158945 0.158315 0.158636 0.158086 0.158436 0.157078 0.154277 0.158785 0.159547 0.157244 0.154449 0.159644 0.153137 0.152874 0.151488 0.156190 H_oc_T355_jigu 4 0.190083 0.219295 0.162588 0.074549 0.000000 0.060745 0.064148 0.065190 0.066273 0.066725 0.063013 0.068867 0.068071 0.061564 0.067716 0.158063 0.155784 0.155199 0.155600 0.156860 0.155153 0.157479 0.155199 0.158636 0.155864 0.156643 0.152547 0.155463 0.156660 0.156305 0.152862 0.154615 0.157153 0.156133 0.157399 0.154014 0.155085 0.157393 0.154191 0.158338 0.155331 0.156958 0.156116 0.153332 0.157084 0.156545 0.156671 0.156752 0.157307 0.156935 0.153286 0.156694 0.156952 0.157055 0.151843 0.157542 0.150611 0.150932 0.149088 0.152874 H_ro_A547_ma 5 0.189619 0.219267 0.161632 0.073765 0.060745 0.000000 0.065780 0.065929 0.065568 0.065242 0.061828 0.070282 0.068100 0.061776 0.068592 0.158298 0.156500 0.154117 0.154712 0.156832 0.153624 0.156488 0.154266 0.157485 0.155388 0.155308 0.151614 0.156064 0.155646 0.155577 0.151906 0.153991 0.156437 0.154203 0.156958 0.152988 0.153086 0.156414 0.153223 0.157061 0.154077 0.156058 0.155010 0.152284 0.156459 0.156184 0.156941 0.155383 0.156855 0.156563 0.152925 0.155955 0.156878 0.155274 0.151436 0.156139 0.149930 0.149849 0.147822 0.152295 H_ro_J213_ma 6 0.185873 0.215773 0.160269 0.076858 0.064148 0.065780 0.000000 0.059582 0.059703 0.058964 0.063122 0.063500 0.062796 0.